In [87]:
import pandas as pd
from datetime import datetime
from datetime import date
import numpy as np

In [88]:
df_sales = pd.read_csv('../data/project/NYC Residential Sales Master.csv', index_col=0)

In [11]:
df_sales.head()

,ADDRESS,BOROUGH,NEIGHBORHOOD,TAX CLASS,BLOCK,LOT,ZIP,RESI UNITS,COMM UNITS,# UNITS,...,LATITUDE,bank,bus_station,gym,hospital,park,restaurant,school,shopping_mall,subway_station
2,2186 BATHGATE AVENUE,2,BATHGATE,1,3048,19,10457,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4443 PARK AVENUE,2,BATHGATE,1,3030,71,10457,2,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4435 PARK AVENUE,2,BATHGATE,1,3030,76,10457,2,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,459 EAST 179 STREET,2,BATHGATE,1,3036,62,10457,2,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,459 EAST 179 STREET,2,BATHGATE,1,3036,62,10457,2,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_sales['SALE DATE'] = df_sales['SALE DATE'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y'))

In [81]:
def category_date(d):
    if date(2009,1,1) <= d <= date(2010,12,31):
        return 'p1'
    if date(2011,1,1) <= d <= date(2012,12,31):
        return 'p2'
    if date(2013,1,1) <= d <= date(2014,12,31):
        return 'p3'
    if date(2015,1,1) <= d <= date(2016,12,31):
        return 'p4'
    if date(2017,1,1) <= d <= date(2018,12,31):
        return 'p5'
    if d >= date(2019,1,1):
        return 'p6'
    return None

dict_p = {
    'p1': '2009-2010',
    'p2': '2011-2012',
    'p3': '2013-2014',
    'p4': '2015-2016',
    'p5': '2017-2018',
    'p6': '2019'
}

In [53]:
df_sales['period'] = df_sales['SALE DATE'].apply(category_date)

In [46]:
groups = [p for p in df_sales['period'].unique() if p is not None]

In [69]:
df_sales_p = df_sales[df_sales['period']=='p1']
df_sales_p = df_sales_p.drop_duplicates(keep='first')\
                       .reset_index(drop=True)\
                       .groupby(['ZIP'])\
                       .agg({'SALE PRICE': ['mean', 'median', 'count']})\
                       .drop(0)

df_sales_p.columns = [f"average sales price {dict_p['p1']}", 
                      f"median sales price {dict_p['p1']}", 
                      f"number of transactions {dict_p['p1']}"]
df_sales_p.reset_index()\
          .astype(dtype={'ZIP': str})\
          .reset_index(drop=True)

,ZIP,average sales price 2009-2010,median sales price 2009-2010,number of transactions 2009-2010
0,10001,3.716200e+06,4150000.0,5
1,10002,2.982263e+06,2600000.0,19
2,10003,5.065943e+06,4300000.0,45
3,10007,9.451570e+06,9451570.0,1
4,10009,2.654923e+06,1450000.0,24
...,...,...,...,...
166,11436,2.899352e+05,283000.0,546
167,11691,3.470800e+05,311552.0,536
168,11692,4.035145e+05,330000.0,211
169,11693,3.087198e+05,299250.0,106


In [82]:
def group_sales(period, df):
    df_sales_p = df[df['period']==period]
    df_sales_p = df.drop_duplicates(keep='first')\
                   .reset_index(drop=True)\
                   .groupby(['ZIP'])\
                   .agg({'SALE PRICE': ['mean', 
                                        'median', 
                                        'count']})

    df_sales_p.columns = [f"average sales price {dict_p[period]}", 
                          f"median sales price {dict_p[period]}", 
                          f"number of transactions {dict_p[period]}"]
    
    df_sales_p = df_sales_p.reset_index()\
                           .astype(dtype={'ZIP': str})\
                           .reset_index(drop=True)
    
    return df_sales_p

In [73]:
group_sales('p1', df_sales)

,ZIP,average sales price 2009-2010,median sales price 2009-2010,number of transactions 2009-2010
0,10001,3.716200e+06,4150000.0,5
1,10002,2.982263e+06,2600000.0,19
2,10003,5.065943e+06,4300000.0,45
3,10007,9.451570e+06,9451570.0,1
4,10009,2.654923e+06,1450000.0,24
...,...,...,...,...
166,11436,2.899352e+05,283000.0,546
167,11691,3.470800e+05,311552.0,536
168,11692,4.035145e+05,330000.0,211
169,11693,3.087198e+05,299250.0,106


In [89]:
def gen_sales_summary(df_sales, export_path):
    df_sales['SALE DATE'] = df_sales['SALE DATE'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y'))
    df_sales['period'] = df_sales['SALE DATE'].apply(category_date)
    
    groups = [p for p in df_sales['period'].unique() if p is not None]
    
    for period in groups:
        df_sales_p = group_sales(period, df_sales)
        df_sales_p.to_csv(f'{export_path}/sales_summary_{period}.csv', index=False)

In [90]:
gen_sales_summary(df_sales, '../data/project/sales_summary')

In [129]:
sales_path = '../data/project/sales_summary'
donations_path = '../data/project/Political_contributions_summary'
sales_summary = pd.read_csv(f'{sales_path}/sales_summary_p1.csv')
donations_summary = pd.read_csv(f'{donations_path}/summary_2009-2010.csv')

In [130]:
sales_summary

,ZIP,average sales price 2009-2010,median sales price 2009-2010,number of transactions 2009-2010
0,0,6.955782e+05,706387.5,30
1,10001,7.864643e+06,6078750.0,68
2,10002,6.234941e+06,5250000.0,207
3,10003,7.963247e+06,5830246.0,328
4,10004,3.938000e+06,2676000.0,4
...,...,...,...,...
172,11436,3.533441e+05,340000.0,2698
173,11691,7.646252e+05,385000.0,2849
174,11692,4.718308e+05,440600.0,1391
175,11693,3.354517e+05,320000.0,672


In [131]:
donations_summary

,zip code,total donations 2009-2010,average donations 2009-2010,median donations 2009-2010,number of donations 2009-2010,state
0,0,1338.0,223.000000,210.0,6,ZZ
1,1,9031.0,1290.142857,500.0,7,NY
2,2,19700.0,2188.888889,2000.0,9,MD
3,3,3000.0,272.727273,250.0,11,MS
4,4,13125.0,252.403846,250.0,52,OH
...,...,...,...,...,...,...
26386,99901,23009.0,418.345455,250.0,55,AK
26387,99921,950.0,316.666667,250.0,3,AK
26388,99928,1950.0,487.500000,375.0,4,AK
26389,99929,1600.0,266.666667,225.0,6,AK


In [134]:
sales_zipcodes = sales_summary['ZIP'].unique()
donations_zipcodes = donations_summary['zip code'].unique()
zipcodes = list(set(sales_zipcodes).intersection(set(donations_zipcodes)))

In [139]:
sales_data = sales_summary.loc[sales_summary.ZIP.isin(zipcodes)][['ZIP', 'average sales price 2009-2010']].reset_index(drop=True)
donations_data = donations_summary.loc[donations_summary['zip code'].isin(zipcodes)][['zip code', 'total donations 2009-2010']].reset_index(drop=True)


In [143]:
sales_data

,ZIP,average sales price 2009-2010
0,0,6.955782e+05
1,10001,7.864643e+06
2,10002,6.234941e+06
3,10003,7.963247e+06
4,10004,3.938000e+06
...,...,...
170,11436,3.533441e+05
171,11691,7.646252e+05
172,11692,4.718308e+05
173,11693,3.354517e+05


In [146]:
data = pd.DataFrame(list(zip(sales_data['average sales price 2009-2010'], 
                     donations_data['total donations 2009-2010'])),
                    index=sales_data['ZIP'])

In [149]:
data

,0,1
ZIP,,
0,6.955782e+05,1338.0
10001,7.864643e+06,287000.0
10002,6.234941e+06,45405.0
10003,7.963247e+06,905278.0
10004,3.938000e+06,721756.0
...,...,...
11436,3.533441e+05,250.0
11691,7.646252e+05,17326.0
11692,4.718308e+05,250.0
